In [1]:
import pandas as pd
import numpy as np

In [2]:
Sheets= pd.read_excel(r"Design1.xlsx", sheet_name=["FMSI", "Kits", "Shim_crossing", "Kit_crossing", "SHIMS"])
locals().update(Sheets)
RFQ=pd.read_excel(r'SHIM_RFQ.xlsx', skiprows = range(1, 12), usecols = "B:H")
cols= RFQ.iloc[0].values
RFQ= RFQ.iloc[1:]
RFQ.columns= cols
RFQ= RFQ[RFQ[cols[0]]=="Shim"]
RFQ['Flat or Tabbed']= RFQ['Flat or Tabbed'].map({'Flat': 'Staked', 'Tabbed': 'Tabbed'})
print(Sheets.keys())
RFQ.head()

dict_keys(['FMSI', 'Kits', 'Shim_crossing', 'Kit_crossing', 'SHIMS'])


,Commodity Name/ Item Description,Flat or Tabbed,Shim Material,Xinlida Material Code,D Plate Number,Estimated Annual Volume,Included in previous 6.24.2022 RFQ?
1,Shim,Staked,CL,S02,"D1329, D1330, D1411, D1565",333984,YES
2,Shim,Staked,CL,S02,"D1313, D1445, D1544, D1594, D1812, D1813, D1848",332823.272727,YES
3,Shim,Tabbed,RRWT,T01,"D430, D653, D815",325191.272727,YES
4,Shim,Tabbed,RRWT,T01,"D976, D1303",303591.272727,YES
5,Shim,Tabbed,RRWT,T01,"D1211, D1293",284718.545455,YES


In [3]:
FMSI= FMSI.astype(str)
Shim_crossing= Shim_crossing.astype(str)

In [4]:
Flat_SHIMS= FMSI.merge(Shim_crossing, on= 'FMSI').merge(SHIMS, on= 'SHIM PN')

In [5]:
FMSI_matches= RFQ['D Plate Number'].str.extractall(r'(D\d+)')
FMSI_matches.rename({0: 'FMSI'}, axis= 1, inplace= True)
FMSI_matches.index.rename(['line', 'match'], inplace= True)
for i in RFQ.index:
    FMSI_matches.loc[i, 'Flat or Tabbed']= RFQ.loc[i, 'Flat or Tabbed']

In [6]:
#to understand logic
First_group= FMSI_matches.loc[1, 'FMSI']
for val in First_group:  #First_group
  Flat_or_tabbed= FMSI_matches[FMSI_matches['FMSI']==val]['Flat or Tabbed'].iloc[0]
  temp_df= Flat_SHIMS[(Flat_SHIMS['FMSI']==val) & (Flat_SHIMS['ATTACHMENT METHOD']==Flat_or_tabbed)]
  temp_df.drop('LINK TYPE', axis= 1).drop_duplicates(inplace= True)
  break

In [58]:
SubResults= []
for line in FMSI_matches.index.get_level_values('line'):
    line_results=[]
    for match in FMSI_matches.loc[line, 'FMSI']:
        Flat_or_tabbed= FMSI_matches[FMSI_matches['FMSI']==val]['Flat or Tabbed'].iloc[0]
        temp_df= Flat_SHIMS[(Flat_SHIMS['FMSI']==match) & (Flat_SHIMS['ATTACHMENT METHOD']==Flat_or_tabbed)]
        temp_df.drop('LINK TYPE', axis= 1).drop_duplicates(inplace= True)
        temp_df= Flat_SHIMS[Flat_SHIMS['FMSI']==match]
        line_results.append(set(temp_df["SHIM PN"].values))
    result= line_results[0].intersection(*line_results)
    Result_df= Flat_SHIMS.loc[(Flat_SHIMS['SHIM PN'].isin(result)) & (Flat_SHIMS['FMSI']==match)]
    Result_df= Result_df.drop('LINK TYPE', axis= 1).drop_duplicates()
    Result_df['Line']= line
    Result_df.set_index(['Line', 'SHIM PN'], inplace= True)
    SubResults.append(Result_df)
    
Output= pd.concat(SubResults)

In [60]:
Flat_SHIMS['SHIM PN']

0       SHM2760-T
1       SHM2760-T
2       SHM1948-S
3       SHM1948-S
4       SHM1948-S
          ...    
4283    SHM5192-T
4284    SHM5199-S
4285    SHM5199-S
4286    SHM5200-T
4287    SHM5200-T
Name: SHIM PN, Length: 4288, dtype: object

In [57]:
match

'D1003'